In [51]:
import sys
import os

# needed to be able to import from src
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.functions import *

In [53]:

dev_set_df=pd.read_csv("../data/assignment1_dev_set.csv")
val_set_df=pd.read_csv("../data/assignment1_val_set.csv")

# print(list(dev_set_df))

bmi=dev_set_df["BMI"]
age=dev_set_df["Host age"]
sex=dev_set_df["Sex"]

to_drop=['Unnamed: 0', 'Project ID','Experiment type','Sex','Host age','Disease MESH ID','BMI']

bacteria=dev_set_df.drop(labels=to_drop,axis=1)

# we need to encode the sexes to 0 and 1
sex_encoded=sex.apply(encode_sex)


# print(sex_encoded)
# print(list(bacteria))
# print(bmi)

# for bacterium in list(bacteria):
#     print(f"--{bacterium}--\n Mean: {bacteria[bacterium].mean()}\n Standard deviation: {bacteria[bacterium].std()}\n \
# Max:{bacteria[bacterium].max()}\n Min:{bacteria[bacterium].min()}")
